In [1]:
#!pip install pygame

In [2]:
import pygame
import numpy as np
#from ball import Ball
#from cart import Cart
#from agent import Agent
#from game import Game

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
window_size = [900, 500]

In [4]:
import random
import math
#Класс мяча
class Ball:
    def __init__(self, surface, position):
        self.radius = 20 
        self.surface = surface
        self.position = position
        self.speed = 4
        self.no_colision = 10
        
        angle = random.randint(-45, 45) + random.randint(0, 1) * 180
        
        self.velocity_x = self.speed * math.cos(angle * math.pi / 180)
        self.velocity_y = self.speed * math.sin(angle * math.pi / 180)
        
        self.color = (255,0,0) #Красный

    def draw(self):  
        pygame.draw.circle(self.surface, 
                           self.color, 
                           (int(self.position[0]), int(self.position[1])), 
                           self.radius)

    def update(self):
        self.position[0] += self.velocity_x
        self.position[1] += self.velocity_y 
        
    def vel_turn(self, angle):
        x = self.velocity_x
        y = self.velocity_y
        self.velocity_x = -math.cos(angle) * x + math.sin(angle) * y
        self.velocity_y = -math.sin(angle) * x - math.cos(angle) * y
        
        
    def set_Position(self, position):
        self.position = position
        
        angle = random.randint(-45, 45) + random.randint(0, 1) * 180
        
        self.velocity_x = self.speed * math.cos(angle * math.pi / 180)
        self.velocity_y = self.speed * math.sin(angle * math.pi / 180)

In [5]:
#Класс тележки
class Cart:
    def __init__(self, surface, position):
        self.velocity = 3
        self.size = [10, 50] 
        self.position = position
        self.surface = surface
        self.color = (0,255,0) 
    
    def draw(self): 
        pygame.draw.rect(self.surface, 
                         self.color, 
                         pygame.Rect(self.position[0],  
                                     self.position[1], 
                                     self.size[0], 
                                     self.size[1]))
    def set_Position(self, position):
        self.position = position

    def move_up(self):
        if self.position[1] <= window_size[1] - self.size[1]:
            self.position[1] += self.velocity
    def move_down(self):
        if self.position[1] > 0:
            self.position[1] -= self.velocity

In [6]:
#Класс вирутального игрока
class Agent():
    def __init__(self, cart):
        self.cart = cart
        self.last = self.cart.move_up
        self.n = -1
    #Алгоритм случайной игры    
    def auto_play(self, ball, cart):
        if self.n > 0:
            self.n -= 1
        else:
            self.n = 10
            if ball.position[1] > cart.position[1]:
                self.last = random.choice((self.cart.move_up, self.cart.move_up, self.cart.move_up))
            else:
                self.last = random.choice((self.cart.move_down, self.cart.move_down, self.cart.move_down))
        self.last()                          

In [7]:
class Game():
    def __init__(self, n_player):
        pygame.init()
        self.window = pygame.display.set_mode(window_size)
        pygame.display.set_caption('Ping Pong')
        #Игра начата
        self.game = True
        self.n_player = n_player
        
        self.win = 100
        self.lose = -300
        self.repel = 10
        
        #Создаю тележки
        self.cart_1 = Cart(self.window, [0, window_size[1] / 2]) 
        self.cart_2 = Cart(self.window, [window_size[0] - 10, window_size[1]/2])
        #Создаю мяч
        self.ball = Ball(self.window, [window_size[0]/2, window_size[1]/2])

        #Создаю агента
        self.agent = Agent(self.cart_2)

        #Рисую мяч и тележки
        self.cart_1.draw()
        self.cart_2.draw()
        self.ball.draw()
        pygame.display.update()
    
    def reset(self):
        #Перезапуск игры
        self.game = True
        
        #Создаю тележки
        self.cart_1 = Cart(self.window, [0, window_size[1] / 2]) 
        self.cart_2 = Cart(self.window, [window_size[0] - 10, window_size[1]/2])
        #Создаю мяч
        self.ball = Ball(self.window, [window_size[0]/2, window_size[1]/2])

        #Создаю агента
        self.agent = Agent(self.cart_2)

        #Рисую мяч и тележки
        self.cart_1.draw()
        self.cart_2.draw()
        self.ball.draw()
        pygame.display.update()
        
        if self.n_player == 1:
            return [*np.divide(self.cart_1.position, window_size), *np.divide(self.ball.position, window_size), self.ball.velocity_x / (2 * self.ball.speed) + 0.5, self.ball.velocity_y / (2 *self.ball.speed) + 0.5]
        else:
            return [*np.divide(self.cart_1.position, window_size), *np.divide(self.cart_2.position, window_size), *np.divide(self.ball.position, window_size), self.ball.velocity_x / (2 *self.ball.speed) + 0.5, self.ball.velocity_y / (2 * self.ball.speed) + 0.5]
        
    # Функция получения текущей ситуации на игровом поле
    
    def game_cicle(self, a):
        r = 0
        r2 = 0
        #Какие кнопки нажаты
        #keys = pygame.key.get_pressed()
            
        #Если нажата вверх
        #if keys[pygame.K_UP]:
        #    self.cart_1.move_down()
        #Если нажата вниз
        #if keys[pygame.K_DOWN]:
        #    self.cart_1.move_up()
        #Если нажата W
        #if keys[pygame.K_w]:
        #    self.cart_2.move_down()
        #Если нажата S
        #if keys[pygame.K_s]:
        #    self.cart_2.move_up()
        #if keys[pygame.K_ESCAPE]:
        #    game = False
        #    pygame.quit()            
        
        if(self.ball.no_colision == 0):
        
            #Если шар касается тележки     
            if (self.cart_2.position[0] - self.ball.position[0] - self.ball.radius <= 0 and
                self.cart_2.position[0] + self.cart_2.size[0]/2 - self.ball.position[0] - self.ball.radius > 0 and 
                self.cart_2.position[1] - self.ball.position[1] < 0 and
                self.ball.position[1] - self.cart_2.position[1] - self.cart_2.size[1] < 0):
            
                self.ball.velocity_x = -1 * self.ball.velocity_x
                r2 += self.repel
                self.ball.no_colision = 20
            
            if (self.cart_1.position[0] + self.cart_1.size[0] - self.ball.position[0] + self.ball.radius >= 0 and
                self.cart_1.position[0] + self.cart_1.size[0]/2 - self.ball.position[0] + self.ball.radius < 0 and
                self.cart_1.position[1] - self.ball.position[1] < 0 and
                self.ball.position[1] - self.cart_1.position[1] - self.cart_1.size[1] < 0):
            
                self.ball.velocity_x = -1 * self.ball.velocity_x
                r += self.repel
                self.ball.no_colision = 20
        
            #Если шар касается верхнего угла 2 тележки
            if (np.linalg.norm(np.subtract(self.cart_2.position, self.ball.position)) <= self.ball.radius and
                np.linalg.norm(np.subtract(self.cart_2.position, self.ball.position)) > self.ball.radius/2 and
                self.cart_2.position[1] - self.ball.position[1] >= 0):
            
                v1 = np.subtract(self.cart_2.position, self.ball.position)
                v2 = self.ball.velocity_x, self.ball.velocity_y
                angle = np.arccos(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))
                if (v1 / np.linalg.norm(v1))[1] < (v2 / np.linalg.norm(v2))[1]:
                    angle *= -1
                self.ball.vel_turn(2 * angle)
                r2 += self.repel
                self.ball.no_colision = 20
        
            #Если шар касается нижнего угла 2 тележки
            if (np.linalg.norm(np.subtract([self.cart_2.position[0], self.cart_2.position[1]+self.cart_2.size[1]], self.ball.position)) <= self.ball.radius and
                np.linalg.norm(np.subtract([self.cart_2.position[0], self.cart_2.position[1]+self.cart_2.size[1]], self.ball.position)) > self.ball.radius/2 and
                self.ball.position[1] - self.cart_2.position[1] - self.cart_2.size[1] >= 0):
            
                v1 = np.subtract( [self.cart_2.position[0], self.cart_2.position[1] + self.cart_2.size[1]], self.ball.position)
                v2 = self.ball.velocity_x, self.ball.velocity_y
                angle = np.arccos(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))
                if (v1 / np.linalg.norm(v1))[1] < (v2 / np.linalg.norm(v2))[1]:
                    angle *= -1
                self.ball.vel_turn(2 * angle)
                r2 += self.repel
                self.ball.no_colision = 20
            
            #Если шар касается верхнего угла 1 тележки
            if (np.linalg.norm(np.subtract([self.cart_1.position[0]+self.cart_1.size[0], self.cart_1.position[1]], self.ball.position)) <= self.ball.radius and
                np.linalg.norm(np.subtract([self.cart_1.position[0]+self.cart_1.size[0], self.cart_1.position[1]], self.ball.position)) > self.ball.radius/2 and
                self.cart_1.position[1] - self.ball.position[1] >= 0):
            
                v1 = np.subtract([self.cart_1.position[0]+ self.cart_1.size[0], self.cart_1.position[1]], self.ball.position)
                v2 = self.ball.velocity_x, self.ball.velocity_y
                angle = np.arccos(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))
                if (v1 / np.linalg.norm(v1))[1] > (v2 / np.linalg.norm(v2))[1]:
                    angle *= -1
                self.ball.vel_turn(2 * angle)
                r += self.repel
                self.ball.no_colision = 20
        
            #Если шар касается нижнего угла 1 тележки
            if (np.linalg.norm(np.subtract([self.cart_1.position[0]+self.cart_1.size[0], self.cart_1.position[1]+self.cart_1.size[1]], self.ball.position)) <= self.ball.radius and
                np.linalg.norm(np.subtract([self.cart_1.position[0]+self.cart_1.size[0], self.cart_1.position[1]+self.cart_1.size[1]], self.ball.position)) > self.ball.radius/2 and
                self.ball.position[1] - self.cart_1.position[1] - self.cart_1.size[1] >= 0):
                
                v1 = np.subtract(np.sum([self.cart_1.position, self.cart_1.size], axis = 1), self.ball.position)
                v2 = self.ball.velocity_x, self.ball.velocity_y
                angle = np.arccos(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))
                if (v1 / np.linalg.norm(v1))[1] > (v2 / np.linalg.norm(v2))[1]:
                    angle *= -1
                self.ball.vel_turn(2 * angle)
                
                self.ball.velocity_x *= -1
                self.ball.velocity_y *= -1
                
                r += self.repel
                self.ball.no_colision = 20
        else:
            self.ball.no_colision -= 1
        
        #Отражение
        if (self.ball.position[1] - self.ball.radius <= 0 or 
            self.ball.position[1] + self.ball.radius >= window_size[1]):
            # изменить направление скорости по y
            self.ball.velocity_y = -1 * self.ball.velocity_y 
        
        
        #Если шар касается сторон экрана для завершения
        if (self.ball.position[0] <= 0):
            self.game = False
            r += self.lose
            r2 += self.win
        if (self.ball.position[0] >= window_size[0]):
            self.game = False
            r += self.win
            r2 += self.lose

        #Переместить шар
        self.ball.update()
        
        #Переместить каретку
        if self.n_player == 1:
            if a == 0:
                self.cart_1.move_up()
            else:
                self.cart_1.move_down()
        else:
            if a[0] == 0:
                self.cart_1.move_up()
            else:
                self.cart_1.move_down()
            if a[1] == 0:
                self.cart_2.move_up()
            else:
                self.cart_2.move_down()
        
        if self.n_player == 1:
            self.agent.auto_play(self.ball, self.cart_2)

        #Очистка экрана
        self.window.fill((0,0,0))
        #Заново рисуем тележку
        self.cart_1.draw()   
        self.cart_2.draw()

        #Заново рисуем шар
        self.ball.draw()
        #Обновляем экран
        pygame.display.update()
        if self.n_player == 1:
            return [*np.divide(self.cart_1.position, window_size), *np.divide(self.ball.position, window_size), self.ball.velocity_x / (2 * self.ball.speed) + 0.5, self.ball.velocity_y / (2 *self.ball.speed) + 0.5], r, self.game
        else:
            return [*np.divide(self.cart_1.position, window_size), *np.divide(self.cart_2.position, window_size), *np.divide(self.ball.position, window_size), self.ball.velocity_x / (2 *self.ball.speed) + 0.5, self.ball.velocity_y / (2 * self.ball.speed) + 0.5], (r, r2), self.game

In [8]:
#while True:
#    game = Game(1)
#    while game.game:
#        action = -1
#        while game.game and action == -1:
#            keys = pygame.key.get_pressed()    
#            #Если нажата вверх
#            if keys[pygame.K_UP]:
#                action = 1
#            #Если нажата вниз
#            if keys[pygame.K_DOWN]:
#                action = 0
#            for event in pygame.event.get():
#                #Выход
#                if event.type == pygame.QUIT:
#                    self.game = False
#                    pygame.quit()
#                if event.type == pygame.KEYDOWN:
#                   # if event.key == pygame.K_UP:
#                   #     action = 1
#                   # if event.key == pygame.K_DOWN:
#                   #     action = 0
#                    if event.key == pygame.K_ESCAPE:
#                        game = False
#                        pygame.quit()
#        game.game_cicle(action)
#        pygame.time.delay(10)
#pygame.quit()

In [ ]:
from sklearn.neural_network import MLPClassifier

n_actions = 2

agent = MLPClassifier(
    hidden_layer_sizes=(20, 20),
    activation='tanh',
    warm_start=True,  # keep progress between .fit(...) calls
    max_iter=1,  # make only 1 iteration on each .fit(...)
)
# initialize agent to the dimension of state an amount of actions

game = Game(1)

agent.fit([game.reset()]*n_actions, range(n_actions))


C:\Users\PM65M\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(20, 20), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=True)

In [ ]:
def generate_session():

    states, actions = [], []
    total_reward = 0

    s = game.reset()

    while True:

        # predict array of action probabilities
        probs = agent.predict_proba([s])[0]

        a = np.random.choice([0,1], p = probs)

        new_s, r, done = game.game_cicle(a)
        
        # record sessions like you did before
        states.append(s)
        actions.append(a)
        total_reward += r

        s = new_s
        if done == False:
            break
    return states, actions, total_reward

In [ ]:
def select_elites(states_batch, actions_batch, rewards_batch, percentile=50):
    """
    Select states and actions from games that have rewards >= percentile
    :param states_batch: list of lists of states, states_batch[session_i][t]
    :param actions_batch: list of lists of actions, actions_batch[session_i][t]
    :param rewards_batch: list of rewards, rewards_batch[session_i]

    :returns: elite_states,elite_actions, both 1D lists of states and respective actions from elite sessions

    Please return elite states and actions in their original order 
    [i.e. sorted by session number and timestep within session]

    If you are confused, see examples below. Please don't assume that states are integers
    (they will become different later).
    """

    reward_threshold = np.percentile(rewards_batch, q = percentile)

    id = np.where(rewards_batch >= reward_threshold)
    
    elite_states = np.concatenate(np.array(states_batch)[id])
    elite_actions = np.concatenate(np.array(actions_batch)[id])

    return elite_states, elite_actions

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import clear_output

def show_progress(rewards_batch, log, percentile, reward_range=[-990, +10]):
    """
    A convenience function that displays training progress. 
    No cool math here, just charts.
    """

    mean_reward = np.mean(rewards_batch)
    threshold = np.percentile(rewards_batch, percentile)
    log.append([mean_reward, threshold])

    clear_output(True)
    print("mean reward = %.3f, threshold=%.3f" % (mean_reward, threshold))
    plt.figure(figsize=[8, 4])
    plt.subplot(1, 2, 1)
    plt.plot(list(zip(*log))[0], label='Mean rewards')
    plt.plot(list(zip(*log))[1], label='Reward thresholds')
    plt.legend()
    plt.grid()

    plt.subplot(1, 2, 2)
    plt.hist(rewards_batch, range=reward_range)
    plt.vlines([np.percentile(rewards_batch, percentile)],
               [0], [100], label="percentile", color='red')
    plt.legend()
    plt.grid()

    plt.show()

In [ ]:
n_sessions = 100
percentile = 70
log = []

for i in range(100):
    # generate new sessions
    sessions = [generate_session() for _ in range(n_sessions)]

    states_batch, actions_batch, rewards_batch = map(np.array, zip(*sessions))

    print((states_batch, actions_batch, rewards_batch))
    
    elite_states, elite_actions = select_elites(states_batch, actions_batch, rewards_batch,  percentile = percentile )

    agent.fit(elite_states, elite_actions)

    show_progress(rewards_batch, log, percentile, reward_range=[0, np.max(rewards_batch)])